In [1]:
import numpy as np
import pandas as pd
import os
print(os.listdir('./'))

['lgbm_importances.png', 'sub_geomean.npy', 'sub_NN.npy', 'Untitled1.ipynb', '.DS_Store', 'masters-final-project', '0513.ipynb', 'test', 'inversion', 'Untitled3.ipynb', 'oof_geomean.npy', 'Untitled.ipynb', 'oof_NN.npy', 'sub_LGBM.npy', 'master2.py', 'oof_LGBM.npy', '0611.ipynb', 'Untitled2.ipynb', 'untitled.txt', 'submission_geomean.csv', 'master.py', 'model.hdf5', 'train.csv', '.ipynb_checkpoints', 'submission_LGBM.csv', 'test_x.csv', 'submission_NN.csv', 'train_y.csv', 'sample_submission.csv', '.idea']


In [2]:
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR
from sklearn.metrics import mean_absolute_error

In [3]:
train = pd.read_csv('./train.csv', dtype={'acoustic_data': np.int16, 'time_to_failure': np.float64})

KeyboardInterrupt: 

In [ ]:
train.head()

In [ ]:
pd.options.display.precision = 15
train.head()

In [ ]:
rows = 150_000
segments = int(np.floor(train.shape[0] / rows))

X_train = pd.DataFrame(index=range(segments), dtype=np.float64,
                       columns=['ave', 'std', 'max', 'min'])
y_train = pd.DataFrame(index=range(segments), dtype=np.float64,
                       columns=['time_to_failure'])

for segment in tqdm(range(segments)):
    seg = train.iloc[segment*rows:segment*rows+rows]
    x = seg['acoustic_data'].values
    y = seg['time_to_failure'].values[-1]
    
    y_train.loc[segment, 'time_to_failure'] = y
    
    X_train.loc[segment, 'ave'] = x.mean()
    X_train.loc[segment, 'std'] = x.std()
    X_train.loc[segment, 'max'] = x.max()
    X_train.loc[segment, 'min'] = x.min()

In [ ]:
X_train.head()

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)

In [ ]:
svm = NuSVR()
svm.fit(X_train_scaled, y_train.values.flatten())
y_pred = svm.predict(X_train_scaled)

In [ ]:
plt.figure(figsize=(6, 6))
plt.scatter(y_train.values.flatten(), y_pred)
plt.xlim(0, 20)
plt.ylim(0, 20)
plt.xlabel('actual', fontsize=12)
plt.ylabel('predicted', fontsize=12)
plt.plot([(0, 0), (20, 20)], [(0, 0), (20, 20)])
plt.show()

In [ ]:
score = mean_absolute_error(y_train.values.flatten(), y_pred)
print(f'Score: {score:0.3f}')

In [ ]:
submission = pd.read_csv('./sample_submission.csv', index_col='seg_id')

In [ ]:
X_test = pd.DataFrame(columns=X_train.columns, dtype=np.float64, index=submission.index)

In [ ]:
for seg_id in X_test.index:
    seg = pd.read_csv('./test/' + seg_id + '.csv')
    
    x = seg['acoustic_data'].values
    
    X_test.loc[seg_id, 'ave'] = x.mean()
    X_test.loc[seg_id, 'std'] = x.std()
    X_test.loc[seg_id, 'max'] = x.max()
    X_test.loc[seg_id, 'min'] = x.min()

In [ ]:
X_test

In [ ]:
X_test_scaled = scaler.transform(X_test)
submission['time_to_failure'] = svm.predict(X_test_scaled)
submission.to_csv('submission0515-1137.csv')